In [1]:
import pandas as pd
import math
import pickle

In [2]:
data = pd.read_excel('IR_Spring2021_ph12_7k.xlsx')

In [3]:
#class below is for holding single id and single url
class holder:
    def __init__(self,ID = 0,url = None):
        self.ID = ID
        self.url = url
    def __eq__(self,obj):
        return str(self.ID)+self.url == str(obj.ID)+obj.url
    def __str__(self):
        return str(self.ID)+' '+self.url


In [4]:
#class below is for holding term frequency and list of holder classess
class termValue:
    def __init__(self):
        self.frequency = 0
        self.holderList = []
    def addHolder(self,newHolder):
        if newHolder not in self.holderList:
            self.holderList.append(newHolder)

In [5]:
#function below is for calculating frequency of each term
def frequencyCalc(myDictionary):
    for i in myDictionary:
        myDictionary[i].frequency = len(myDictionary[i].holderList)

In [6]:
"""
#algorithm below is for retrieval of the dictionary
myFile = open('dictionary.txt',encoding = 'UTF-8')
theDictionary = {}
for i in myFile:
    res = i.split('$')
    if res[0] not in theDictionary:
        theDictionary[res[0]] = termValue()
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
    else:
        myHolder = holder(res[1],res[2])
        theDictionary[res[0]].addHolder(myHolder)
myFile.close()
"""

"\n#algorithm below is for retrieval of the dictionary\nmyFile = open('dictionary.txt',encoding = 'UTF-8')\ntheDictionary = {}\nfor i in myFile:\n    res = i.split('$')\n    if res[0] not in theDictionary:\n        theDictionary[res[0]] = termValue()\n        myHolder = holder(res[1],res[2])\n        theDictionary[res[0]].addHolder(myHolder)\n    else:\n        myHolder = holder(res[1],res[2])\n        theDictionary[res[0]].addHolder(myHolder)\nmyFile.close()\n"

In [7]:
"""
#algorithm above was inefficient it took about 5 minutes so saving and retrival using pickle is better
pickle.dump(theDictionary,open('dictionary','ab'))
"""

"\n#algorithm above was inefficient it took about 5 minutes so saving and retrival using pickle is better\npickle.dump(theDictionary,open('dictionary','ab'))\n"

In [8]:
#code below is for retrival our dictionary
theDictionary = pickle.load(open('dictionary','rb'))

In [9]:
frequencyCalc(theDictionary)

In [10]:
#list below is just for saving list of words
wordsList = []
for i in theDictionary:
    wordsList.append(i)

In [11]:
#normalize 1 method
def normalize1(myList):
    """"
    we should remove some specific characters eg: we do not save dot in the end of sentence with the last word
    we just need to remove if these specific chars are at the last of our word
    """
    for i in range(len(myList)):
        if myList[i].endswith('.'):
            myList[i] = myList[i].replace('.','')
        elif myList[i].endswith('،'):
            myList[i] = myList[i].replace('،','')
        elif myList[i].endswith(':'):
            myList[i] = myList[i].replace(':','')
        else:
            pass
    #the two last members means end of message and it is in all docs so we do not need it
    myList.pop()
    myList.pop()

In [12]:
#algorithm below is saving documents and their bag of words in dictionary docid as key and list of words as value
bagOfWordsDic = {}
for i in range(len(data)):
    docId = data['id'][i]
    listOfwords = data['content'][i].split()
    normalize1(listOfwords)
    bagOfWordsDic[docId] = wordsList

In [13]:
#function below is for calculating tf
def tf_calculator(term,document):
    documentBagOfWords = document.split()
    normalize1(documentBagOfWords)
    n = documentBagOfWords.count(term)
    if n == 0:
        return 0
    return 1+math.log(n)

In [14]:
#function below is for calculating tf for a query
def tf_calculatorV02(term,query):
    queryBagOfWords = query.split()
    n = queryBagOfWords.count(term)
    if n == 0:
        return 0
    return 1+math.log(n)

In [15]:
#function below is for calculating idf
def idf_calculator(term,N,myDictionary):
    return math.log(N/myDictionary[term].frequency)

In [16]:
#function below is for calculating vector of a single document
def createVector(document,myDictionary,N):
    documentBagOfWords = document.split()
    normalize1(documentBagOfWords)
    resDict = {}
    for i in documentBagOfWords:
        resDict[i] = tf_calculator(i,document)*idf_calculator(i,N,myDictionary)
    return resDict

In [17]:
#function below is for calculating vector for a query
def createVectorV02(query,myDictionary,N):
    queryBagOfWords = query.split()
    resDict = {}
    for i in queryBagOfWords:
        resDict[i] = tf_calculatorV02(i,query)
    return resDict

In [18]:
"""
#algorithm below is for calculating all vector spaces
vectorDictionary = {}
for i in range(len(data)):
    documentId = data['id'][i]
    documentUrl = data['url'][i]
    documentContent = data['content'][i]
    vectorDictionary[str(documentId)+' '+documentUrl] = createVector(documentContent,theDictionary,len(data))
"""

"\n#algorithm below is for calculating all vector spaces\nvectorDictionary = {}\nfor i in range(len(data)):\n    documentId = data['id'][i]\n    documentUrl = data['url'][i]\n    documentContent = data['content'][i]\n    vectorDictionary[str(documentId)+' '+documentUrl] = createVector(documentContent,theDictionary,len(data))\n"

In [19]:
#saving the vector dictionary
#pickle.dump(vectorDictionary,open('vectorDictionary','ab'))

In [20]:
#loading vector dictionary
vectorDictionary = pickle.load(open('vectorDictionary','rb'))

In [21]:
#function below is for calculating size
def size_calculate(vector):
    res = 0
    for i in vector:
        res += math.pow(vector[i],2)
    return math.sqrt(res)

In [22]:
#class below is for saving vectorid + similarity to a query
class vectorSimilarity:
    def __init__(self,similarityValue,vectorId):
        self.similarityValue = similarityValue
        self.vectorId = vectorId
    def __str__(self):
        return self.vectorId
    def __lt__(self,obj):
        return self.similarityValue < obj.similarityValue
    def __eq__(self,obj):
        return self.vectorId == obj.vectorId

In [23]:
#calculating similarity between a document and query
def cosineSimilarity(query,ID,URL,documentVectors,N):
    queryVector = createVectorV02(query,theDictionary,N)
    documentVector = documentVectors[str(ID)+' '+URL]
    res = 0
    for i in queryVector:
        if i in documentVector:
            res += queryVector[i]*documentVector[i]
    return res/size_calculate(documentVector)

In [24]:
#engine below does not use index elimination or any champions list
def engineV01():
    query = input('enter query: ')
    resList = []
    for i in range(len(data)):
        similarity = cosineSimilarity(query,data['id'][i],data['url'][i],vectorDictionary,len(data))
        resList.append(vectorSimilarity(similarity,str(data['id'][i])+' '+data['url'][i]))
    resList = sorted(resList,reverse = True)
    for i in range(10):
        print(resList[i])

In [67]:
#testing engine V01
engineV01()

enter query: تیم پرسپولیس
1032 https://www.isna.ir/news/99122821853/۵-درصد-از-مطالبات-بازیکنان-پرسپولیس-پرداخت-شد
126 https://www.isna.ir/news/99030402072/مثبت-شدن-تست-کرونای-دو-عضو-تدارکات-پرسپولیس
435 https://www.isna.ir/news/99061209143/احسان-پهلوان-به-پرسپولیس-پیوست
1372 https://www.isna.ir/news/98071108502/امید-عالیشاه-به-لیست-پرسپولیس-برگشت
892 https://www.isna.ir/news/99110705179/برگزاری-جلسه-هیات-مدیره-پرسپولیس-در-حضور-اعضای-جدید-و-گل-محمدی
1651 https://www.isna.ir/news/98120302460/بیرانوند-دوباره-نیمکت-نشین-شد
1234 https://www.isna.ir/news/98050401898/مهدی-شریفی-از-لیست-پرسپولیس-خارج-شد
810 https://www.isna.ir/news/99101208800/پرسپولیس-۱۲۰-هزار-دلار-بابت-انتقال-رسن-دریافت-کرد
697 https://www.isna.ir/news/99082719104/مهدی-خانی-با-پرسپولیس-قرارداد-۲ساله-بست
1648 https://www.isna.ir/news/98120201214/درگیری-فیزیکی-در-تمرین-پرسپولیس


In [25]:
#function below is for getting terms from query that exist in our dictionary
def queryDictionarySharing(query,myDictionary):
    queryWords = query.split()
    res = []
    for i in queryWords:
        if i in myDictionary and i not in res:
            res.append(i)
    return res

In [26]:
#function below is for engine when it use index elimination tecnique
#for this i consider only checking documents that having at least one query term
def engineV02():
    query = input('enter query: ')
    resDocs = []
    sharedWords = queryDictionarySharing(query,theDictionary)
    if len(sharedWords) == 0:
        print('no result')
        return 
    for i in sharedWords:
        holderList = theDictionary[i].holderList
        for j in holderList:
            if j not in resDocs:
                resDocs.append(j)
    finalRes = []
    for i in resDocs:
        Id = i.ID
        url = i.url
        similarity = cosineSimilarity(query,Id,url.replace('\n',''),vectorDictionary,len(data))
        finalRes.append(vectorSimilarity(similarity,str(Id)+' '+url))
    finalRes = sorted(finalRes,reverse = True)
    if len(finalRes) >= 10:
        for i in range(10):
            print(finalRes[i])
    else:
        for i in finalRes:
            print(i)

In [66]:
engineV02()

enter query: تیم پرسپولیس
1032 https://www.isna.ir/news/99122821853/۵-درصد-از-مطالبات-بازیکنان-پرسپولیس-پرداخت-شد

126 https://www.isna.ir/news/99030402072/مثبت-شدن-تست-کرونای-دو-عضو-تدارکات-پرسپولیس

435 https://www.isna.ir/news/99061209143/احسان-پهلوان-به-پرسپولیس-پیوست

1372 https://www.isna.ir/news/98071108502/امید-عالیشاه-به-لیست-پرسپولیس-برگشت

892 https://www.isna.ir/news/99110705179/برگزاری-جلسه-هیات-مدیره-پرسپولیس-در-حضور-اعضای-جدید-و-گل-محمدی

1651 https://www.isna.ir/news/98120302460/بیرانوند-دوباره-نیمکت-نشین-شد

1234 https://www.isna.ir/news/98050401898/مهدی-شریفی-از-لیست-پرسپولیس-خارج-شد

810 https://www.isna.ir/news/99101208800/پرسپولیس-۱۲۰-هزار-دلار-بابت-انتقال-رسن-دریافت-کرد

697 https://www.isna.ir/news/99082719104/مهدی-خانی-با-پرسپولیس-قرارداد-۲ساله-بست

1648 https://www.isna.ir/news/98120201214/درگیری-فیزیکی-در-تمرین-پرسپولیس



In [27]:
#class below is for holding single id and single url and the weight of term and this is for champion dictionary
class holderV02:
    def __init__(self,ID = 0,url = None,weight = 0):
        self.ID = ID
        self.url = url
        self.weight = weight
    def __eq__(self,obj):
        return str(self.ID)+self.url == str(obj.ID)+obj.url
    def __str__(self):
        return str(self.ID)+' '+self.url
    def __lt__(self,obj):
        return self.weight < obj.weight


In [35]:
"""
#function below if for creating our dictionary
def createChampionDictionary(myData):
    #dictionary data structure below is for holding term and frequency and holderLists
    #finally our form is like {term:(freq,[listOfHolders])}
    #dictionary[term] is termValue class object that contains freq+lisOfHolders that is object of holder class
    championDictionary = {}
    for counter in range(len(myData)):
        x = myData['content'][counter].split()
        y = myData['id'][counter]
        z = myData['url'][counter]
        normalize1(x)
        for i in x:
            if i not in championDictionary:
                championDictionary[i] = termValue()
                myHolder = holderV02(y,z,x.count(i))
                championDictionary[i].addHolder(myHolder)
            else:
                myHolder = holderV02(y,z,x.count(i))
                championDictionary[i].addHolder(myHolder)
    for i in championDictionary:
        championDictionary[i].holderList = sorted(championDictionary[i].holderList,reverse = True)
        if len(championDictionary[i].holderList) > 10:
            championDictionary[i].holderList = championDictionary[i].holderList[:10]
    return championDictionary
"""

"\n#function below if for creating our dictionary\ndef createChampionDictionary(myData):\n    #dictionary data structure below is for holding term and frequency and holderLists\n    #finally our form is like {term:(freq,[listOfHolders])}\n    #dictionary[term] is termValue class object that contains freq+lisOfHolders that is object of holder class\n    championDictionary = {}\n    for counter in range(len(myData)):\n        x = myData['content'][counter].split()\n        y = myData['id'][counter]\n        z = myData['url'][counter]\n        normalize1(x)\n        for i in x:\n            if i not in championDictionary:\n                championDictionary[i] = termValue()\n                myHolder = holderV02(y,z,x.count(i))\n                championDictionary[i].addHolder(myHolder)\n            else:\n                myHolder = holderV02(y,z,x.count(i))\n                championDictionary[i].addHolder(myHolder)\n    for i in championDictionary:\n        championDictionary[i].holderList

In [36]:
#championDictionary = createChampionDictionary(data)

In [37]:
#pickle.dump(championDictionary,open('championDictionary','ab'))

In [34]:
championDictionary = pickle.load(open('championDictionary','rb'))

In [39]:
#function below is for engine when it use index elimination and champion list tecnique
#for this i consider only checking documents that having at least one query term and r for champion is 10
def engineV03():
    query = input('enter query: ')
    resDocs = []
    sharedWords = queryDictionarySharing(query,championDictionary)
    if len(sharedWords) == 0:
        print('no result')
        return 
    for i in sharedWords:
        holderList = championDictionary[i].holderList
        for j in holderList:
            if j not in resDocs:
                resDocs.append(j)
    finalRes = []
    for i in resDocs:
        Id = i.ID
        url = i.url
        similarity = cosineSimilarity(query,Id,url.replace('\n',''),vectorDictionary,len(data))
        finalRes.append(vectorSimilarity(similarity,str(Id)+' '+url))
    finalRes = sorted(finalRes,reverse = True)
    if len(finalRes) >= 10:
        for i in range(10):
            print(finalRes[i])
    else:
        for i in finalRes:
            print(i)

In [40]:
engineV03()

enter query: تیم پرسپولیس
510 https://www.isna.ir/news/99070604626/خان-محمدی-تعویض-های-گل-محمدی-روند-بازی-با-السد-را-تغییر-داد
1530 https://www.isna.ir/news/98100302640/گله-کالدرون-از-منتقد-سابقش
582 https://www.isna.ir/news/99072418213/در-آخرین-جلسه-مدیران-پرسپولیس-و-بشار-رسن-چه-گذشت
1467 https://www.isna.ir/news/98090200906/اختلاف-شجاع-خلیل-زاده-و-کالدرون-بر-سر-دفاع-راست
309 https://www.isna.ir/news/99042921462/فنونی-زاده-پرسپولیس-خوب-بازی-نمی-کند-اما-می-برد-خطر-کسر-امتیاز
579 https://www.isna.ir/news/99072418443/بشار-رسن-باز-هم-با-هواداران-پرسپولیس-خداحافظی-کرد
835 https://www.isna.ir/news/99102317552/تیم-های-لیگ-برتری-هندبال-خوزستان-در-آستانه-انحلال
208 https://www.isna.ir/news/99032918684/برانکو-پیشنهاد-تسویه-حساب-پرسپولیس-را-رد-کرد-۱۰۴-هزار-فرانک
381 https://www.isna.ir/news/99052720182/مشکلات-عجیب-پرسپولیس-در-پرداخت-مطالبات-مربیان-سابقش
1269 https://www.isna.ir/news/98051508035/برنامه-کامل-نوزدهمین-دوره-لیگ-برتر
